In [42]:
import os
import warnings
import json
import boto3
import pickle
import pandas as pd
import numpy as np
import dask.bag as db
import dask.dataframe as dd
import tensorflow as tf
import keras.applications.inception_resnet_v2 as inception_resnet
from keras import backend
from keras.models import Model, Input, Sequential, load_model
from keras.preprocessing.image import load_img, img_to_array
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Input, Conv2D
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau)
from PIL import Image
from operator import itemgetter
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from dask.distributed import Client
from dask.diagnostics import ProgressBar
from tqdm.notebook import tqdm

In [2]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")


Tensor Flow Version: 2.4.0
Keras Version: 2.4.0
GPU is available


In [3]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
from IPython.display import display, clear_output
from IPython.display import Audio
from IPython.core.display import HTML
import numpy as np

def alert(duration=2):
    """ makes sound on client using javascript (works with remote server) """      
    framerate = 44100
    freq=300
    t = np.linspace(0,duration,framerate*duration)
    data = np.sin(2*np.pi*freq*t)
    display(Audio(data,rate=framerate, autoplay=True))

# Constant Variables

In [5]:
BUCKET = 'kapeles'
DATA_PATH ='BDCC_Dataset/downloads/manual'
TRAIN_PATH =  os.path.join(DATA_PATH, 'train')
TEST_PATH = os.path.join(DATA_PATH, 'test')
SAMPLE_PATH = os.path.join(DATA_PATH, 'sample')

VECTOR_PATH = 'BDCC_Dataset/inception_resnet_vectors'
TARGET_SIZE = (1024, 1024)

# Extract image vectors based on InceptionResNetV2

## Create Inception Resnset V2 base

In [6]:
# Inception Resnet V2 base without top layer and specified input shape
base = inception_resnet.InceptionResNetV2(weights='imagenet',
                                          include_top=False,
                                          input_shape=(TARGET_SIZE +
                                                       tuple([3])),
                                          pooling='avg')
# Freeze layers
for layer in base.layers:
    layer.trainable=False
base.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1024, 1024,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 511, 511, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 511, 511, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 511, 511, 32) 0           batch_normalization[0][0]        
________________________________________________________________________________

## Predict each images using Inception Resnet V2 base

In [7]:
s3 = boto3.resource('s3', region_name='us-east-2')
bucket = s3.Bucket(BUCKET)

In [7]:
def get_image(key, bucket):
    """Return numpy array of image from the s3 bucket"""
    obj = bucket.Object(key)
    response = obj.get()
    file_stream = response['Body']
    im = Image.open(file_stream).resize(TARGET_SIZE)
    return np.array(im)
    

def write_json_file(data, filename):
    """Write json file in the VECTOR_PATH"""
    key = os.path.join(VECTOR_PATH, filename)
    (s3.Object(BUCKET, key)
     .put(Body=bytes(json.dumps(data).encode('UTF-8'))))
    

def read_json_file(filename):
    """Reads json file in the VECTOR_PATH"""
    obj = bucket.Object(os.path.join(VECTOR_PATH, filename))
    response = obj.get()
    file_content = response['Body'].read()
    return json.loads(file_content)

In [ ]:
file_no = 100
train_images = list(bucket.objects.filter(Prefix=TRAIN_PATH))
count = len(train_images)
print('Extracting image vectors for {} images'.format(count))
images = []
for idx in tqdm(range(count)):
    obj = train_images[idx]
    try:
        # Get the numpy matrix of the image
        dct = dict()
        img = get_image(obj.key, bucket)
        
        # Preprocess input based on the training of convolutional base
        nimage = inception_resnet.preprocess_input(img)
        nimage = np.expand_dims(nimage, axis=0)
        
        # Extracted image vectors
        image_vector = base.predict(nimage)
        image_vector = np.reshape(image_vector, image_vector.shape[1:])
        
        dct["name"] = os.path.basename(obj.key)
        dct["value"] = image_vector.tolist()
        images.append(dct)
        
        # Check the image is in the hundredth or the last image
        if (not file_no // (idx + 1)) or count == (idx + 1):
            file_name = f'train_{file_no//100}.json'
            write_json_file(images, file_name)
            file_no += 100
            images = []
    except Exception as e:
        alert(5)
        print('Unable to read {}:{}'.format(idx, obj.key)) 

Extracting image vectors for 35127 images


In [ ]:
file_no = 100
test_images = list(bucket.objects.filter(Prefix=TEST_PATH))
count = len(test_images)
print('Extracting image vectors for {} images'.format(count))
images = []
for idx in tqdm(range(count)):
    obj = test_images[idx]
    try:
        # Get the numpy matrix of the image
        dct = dict()
        img = get_image(obj.key, bucket)
        
        # Preprocess input based on the training of convolutional base
        nimage = inception_resnet.preprocess_input(img)
        nimage = np.expand_dims(nimage, axis=0)
        
        # Extracted image vectors
        image_vector = base.predict(nimage)
        image_vector = np.reshape(image_vector, image_vector.shape[1:])
        
        dct["name"] = os.path.basename(obj.key)
        dct["value"] = image_vector.tolist()
        images.append(dct)
        
        # Check the image is in the hundredth or the last image
        if (not file_no // (idx + 1)) or count == (idx + 1):
            file_name = f'test_{file_no//100}.json'
            write_json_file(images, file_name)
            file_no += 100
            images = []
    except Exception as e:
        alert(5)
        print('Unable to read {}:{}'.format(idx, obj.key)) 

In [56]:
file_no = 100
sample_images = list(bucket.objects.filter(Prefix=SAMPLE_PATH))
count = len(sample_images)
print('Extracting image vectors for {} images'.format(count))
images = []
for idx in tqdm(range(count)):
    obj = sample_images[idx]
    try:
        # Get the numpy matrix of the image
        dct = dict()
        img = get_image(obj.key, bucket)
        
        # Preprocess input based on the training of convolutional base
        nimage = inception_resnet.preprocess_input(img)
        nimage = np.expand_dims(nimage, axis=0)
        
        # Extracted image vectors
        image_vector = base.predict(nimage)
        image_vector = np.reshape(image_vector, image_vector.shape[1:])
        
        dct["name"] = os.path.basename(obj.key)
        dct["value"] = image_vector.tolist()
        images.append(dct)
        
        # Check the image is in the hundredth or the last image
        if (not file_no // (idx + 1)) or count == (idx + 1):
            file_name = f'sample_{file_no//100}.json'
            write_json_file(images, file_name)
            file_no += 100
            images = []
    except Exception as e:
        alert(5)
        print('Unable to read {}:{}'.format(idx, obj.key)) 

Extracting image vectors for 11 images


Unable to read 10:BDCC_Dataset/downloads/manual/sampleSubmission.csv



# Setup

## Create Dask Cluster

In [8]:
# Create dask cluster
client = Client('172.31.31.30:8786')

# Register progress bar
pbar = ProgressBar()
pbar.register()

# Get AWS Config
with open('setting_aws.pkl', 'rb') as f:
    settings_aws = pickle.load(f)

# Show cluster
client

Client Scheduler: tcp://172.31.31.30:8786 Dashboard: http://172.31.31.30:8787/status,Cluster Workers: 2 Cores: 12 Memory: 50.47 GB


# Prepare dataset

In [217]:
test_label_path = ('s3://kapeles/BDCC_Dataset/downloads/'
                   'kagg-foru-mess-atta_9052_2877_reti_solupq'
                   '89ZoBOgYh9qJuSYbz7faT1us7bBw8bXbg_qHmrYMk.csv')
train_label_path = ('s3://kapeles/BDCC_Dataset/downloads/manual'
                    '/trainLabels.csv')
train_label = dd.read_csv(train_label_path, storage_options=settings_aws)
test_label = dd.read_csv(test_label_path,
                         usecols=['image', 'level'],
                         storage_options=settings_aws)

all_labels = train_label.append(test_label).compute()
all_labels.image = all_labels.image.apply(lambda x: str(x)+'.jpeg')
del train_label_path, test_label_path, train_label, test_label

## Split dataset 60 20 20

In [218]:
x_train, x_test, y_train, y_test = train_test_split(all_labels.image,
                                                    all_labels.level,
                                                    test_size=0.2,
                                                    random_state=2021)
train_labels = all_labels.loc[all_labels.image.isin(x_train)]
x_train, x_val, y_train, y_val = train_test_split(train_labels.image,
                                                  train_labels.level,
                                                  test_size=0.25,
                                                  random_state=2021)

del all_labels, train_labels

## Perform sampling to solve imbalance data

In [219]:
n_samples = int(y_train.value_counts().mean())
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
new_x_train, new_y_train = pd.Series(dtype='object'), pd.Series(dtype='int32')
for cls in y_train.unique():
    idx = y_train[y_train==cls].index
    x = x_train.loc[idx]
    y = y_train.loc[idx]
    
    new_x, new_y = resample(x, y, replace=True, n_samples=n_samples,
                            random_state=2021)
    new_x_train = new_x_train.append(new_x)
    new_y_train = new_y_train.append(new_y)

x_train, y_train = new_x_train, new_y_train
del new_x_train, new_y_train

In [220]:
y_train.value_counts()

4    10644
3    10644
2    10644
1    10644
0    10644
dtype: int64

## Create Generator

Create a data generator to get batches from train data.

In [158]:
def data_generator(captions, data, batch_size):
    """
    Create a generator that for 2 inputs (image vector, caption sequence)
    output is the next word in the sequence

    Parameters
    ----------
        captions : dict
            image filename as key and level as value
        data : dask bag
            image filaname as key and vectors from convolutional base as value
        img_map : list or iterator
            list or iterator to include, exclude data not in the img_map
        batch_size : int
            batch_size
    
    Returns
    -------
        result : 2 tuples
            first element is the 2 inputs image vector from the convolutional
            base and text to sequence of the caption
            second element is the output is the next word in the sequence
            ([image_vector, sequence], next_word)
    """
    x, y = [], []
    n = 0
    while 1:
        for image_name, vector in data.items():
            # Get image vector based on the convolutional base
            target = captions[image_name]
            # Set the value of target to binary
            target = to_categorical([target], num_classes=5)[0]
            x.append(vector)
            y.append(target)
            n += 1
            if n == batch_size:
                yield np.array(x), np.array(y)
                x, y = [], []
                n = 0

## Read data using Dask Bag

In [10]:
def join_dict(x, y):
    """Update dictionary x with values of dictionary y"""
    x.update(y)
    return x

In [221]:
# Read training data from S3
data = (db.read_text(os.path.join('s3://', BUCKET, VECTOR_PATH, '*.json'),
                     storage_options=settings_aws)
        .map(json.loads)
        .flatten()
        .persist())

In [ ]:
train_data = (data.filter(lambda x: x['name'] in x_train.values)
              .map(lambda x: {x['name']: np.asarray(x['value'])})
              .fold(binop=lambda x, y: join_dict(x, y),
                    combine=lambda x, y: join_dict(x, y))
              .compute())

In [ ]:
val_data = (data.filter(lambda x: x['name'] in x_val.values)
            .map(lambda x: {x['name']: np.asarray(x['value'])})
            .fold(binop=lambda x, y: join_dict(x, y),
                  combine=lambda x, y: join_dict(x, y))
            .compute())

In [ ]:
# Make train labels to dictionary for easier implementation
train_labels = pd.Series(y_train.values,index=x_train.values).to_dict()
val_labels = pd.Series(y_val.values, index=x_val.values).to_dict()

# Training the model

## Playing with Batch size

In [183]:
def create_toplayers():
    """Create top layer using inceptionb resnet v2 base"""
    # Create input layer based on the output of the convolutional base
    lyr_input = Input(shape=(base.layers[-1].output.shape.as_list()[1], ))

    # Add dropout layer
    lyr_dropout1 = Dropout(0.25)(lyr_input)
    # Add Dense
    lyr_dense1 = Dense(254, activation='relu')(lyr_dropout1)
    # Add dropout layer
    lyr_dropout2 = Dropout(0.25)(lyr_dense1)

    # Create output layer
    output = Dense(5, activation='softmax')(lyr_dropout2)

    model = Model(inputs=[lyr_input], outputs=[output])
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

### Batch size = 512

In [184]:
batch_size = 512
train_gen = data_generator(train_labels, train_data, batch_size)
val_gen = data_generator(val_labels, val_data, batch_size)
train_steps = len(train_labels) // batch_size
val_steps = len(val_labels) // batch_size

In [185]:
checkpoint = ModelCheckpoint('tune/batch_size512.h5',
                             monitor='val_accuracy',
                             verbose=1, save_best_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_accuracy',
                                   factor=0.7,
                                   patience=3,
                                   cooldown=3,
                                   verbose=1,
                                   min_lr=0.0001)
early = EarlyStopping(monitor="val_accuracy", 
                      verbose=1,
                      patience=30)

In [186]:
model = create_toplayers()
model.fit(train_gen,
          steps_per_epoch=train_steps,
          validation_data = val_gen,
          validation_steps = val_steps,
          epochs=200,
          callbacks=[checkpoint, reduceLROnPlat, early])

Epoch 1/200
21/21 [==============================] - 1s 32ms/step - loss: 1.6028 - accuracy: 0.2820 - val_loss: 1.1985 - val_accuracy: 0.6877

Epoch 00001: val_accuracy improved from -inf to 0.68773, saving model to tune/batch_size512.h5
Epoch 2/200
21/21 [==============================] - 1s 26ms/step - loss: 1.4015 - accuracy: 0.3633 - val_loss: 1.1657 - val_accuracy: 0.5588

Epoch 00002: val_accuracy did not improve from 0.68773
Epoch 3/200
21/21 [==============================] - 1s 26ms/step - loss: 1.3449 - accuracy: 0.3763 - val_loss: 1.0741 - val_accuracy: 0.6900

Epoch 00003: val_accuracy improved from 0.68773 to 0.69003, saving model to tune/batch_size512.h5
Epoch 4/200
21/21 [==============================] - 1s 26ms/step - loss: 1.3105 - accuracy: 0.3977 - val_loss: 1.0738 - val_accuracy: 0.6621

Epoch 00004: val_accuracy did not improve from 0.69003
Epoch 5/200
21/21 [==============================] - 1s 26ms/step - loss: 1.2800 - accuracy: 0.4103 - val_loss: 1.0333 - val_

### Batch size = 80

In [33]:
batch_size=80
train_gen = data_generator(labels, train_data, batch_size)
steps = len(train_images)//batch_size

In [35]:
checkpoint = ModelCheckpoint('tune/batch_size80.h5', monitor='loss',
                             verbose=0, save_best_only=True, mode='min')

reduceLROnPlat = ReduceLROnPlateau(monitor='loss',
                                   factor=0.7,
                                   patience=3,
                                   mode='min',
                                   cooldown=3,
                                   verbose=1,
                                   min_lr=0.0001)
early = EarlyStopping(monitor="loss", 
                      mode="min",
                      verbose=1,
                      patience=10)

In [36]:
model = create_toplayers()
model.fit_generator(train_gen,
                    steps_per_epoch=steps,
                    epochs=200,
                    callbacks=[checkpoint, reduceLROnPlat, early])

Epoch 1/200
439/439 [==============================] - 1s 3ms/step - loss: 0.8355 - categorical_accuracy: 0.7328
Epoch 2/200
439/439 [==============================] - 1s 3ms/step - loss: 0.7799 - categorical_accuracy: 0.7370
Epoch 3/200
439/439 [==============================] - 1s 3ms/step - loss: 0.7666 - categorical_accuracy: 0.7406
Epoch 4/200
439/439 [==============================] - 1s 3ms/step - loss: 0.7580 - categorical_accuracy: 0.7440
Epoch 5/200
439/439 [==============================] - 1s 3ms/step - loss: 0.7525 - categorical_accuracy: 0.7451
Epoch 6/200
439/439 [==============================] - 1s 3ms/step - loss: 0.7487 - categorical_accuracy: 0.7473
Epoch 7/200
439/439 [==============================] - 1s 3ms/step - loss: 0.7429 - categorical_accuracy: 0.7487
Epoch 8/200
439/439 [==============================] - 1s 3ms/step - loss: 0.7414 - categorical_accuracy: 0.7489
Epoch 9/200
439/439 [==============================] - 1s 3ms/step - loss: 0.7412 - categorical_

## Remove first Dropout layer

In [178]:
def create_toplayers():
    """Create top layer using inceptionb resnet v2 base"""
    # Create input layer based on the output of the convolutional base
    lyr_input = Input(shape=(base.layers[-1].output.shape.as_list()[1], ))

    # Add Dense
    lyr_dense1 = Dense(254, activation='relu')(lyr_input)
    # Add dropout layer
    lyr_dropout2 = Dropout(0.25)(lyr_dense1)

    # Create output layer
    output = Dense(5, activation='softmax')(lyr_dropout2)

    model = Model(inputs=[lyr_input], outputs=[output])
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                  metrics = ['categorical_accuracy'])
    return model

In [179]:
batch_size = 512
train_gen = data_generator(train_labels, train_data, batch_size)
val_gen = data_generator(val_labels, val_data, batch_size)
train_steps = len(train_labels) // batch_size
val_steps = len(val_labels) // batch_size

In [180]:
checkpoint = ModelCheckpoint('tune/remove_dropout1.h5',
                             monitor='val_categorical_accuracy',
                             verbose=1, save_best_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_categorical_accuracy',
                                   factor=0.7,
                                   patience=3,
                                   cooldown=3,
                                   verbose=1,
                                   min_lr=0.0001)
early = EarlyStopping(monitor="val_categorical_accuracy", 
                      verbose=1,
                      patience=30)

In [181]:
model = create_toplayers()
model.fit(train_gen,
          steps_per_epoch=train_steps,
          validation_data = val_gen,
          validation_steps = val_steps,
          epochs=200,
          callbacks=[checkpoint, reduceLROnPlat, early])

Epoch 1/200
21/21 [==============================] - 1s 33ms/step - loss: 1.5530 - categorical_accuracy: 0.2861 - val_loss: 1.2968 - val_categorical_accuracy: 0.3173

Epoch 00001: val_categorical_accuracy improved from -inf to 0.31733, saving model to tune/remove_dropout1.h5
Epoch 2/200
21/21 [==============================] - 1s 26ms/step - loss: 1.3928 - categorical_accuracy: 0.3599 - val_loss: 1.1665 - val_categorical_accuracy: 0.5289

Epoch 00002: val_categorical_accuracy improved from 0.31733 to 0.52889, saving model to tune/remove_dropout1.h5
Epoch 3/200
21/21 [==============================] - 1s 26ms/step - loss: 1.3266 - categorical_accuracy: 0.3858 - val_loss: 1.1033 - val_categorical_accuracy: 0.6675

Epoch 00003: val_categorical_accuracy improved from 0.52889 to 0.66745, saving model to tune/remove_dropout1.h5
Epoch 4/200
21/21 [==============================] - 1s 27ms/step - loss: 1.2897 - categorical_accuracy: 0.4021 - val_loss: 1.0167 - val_categorical_accuracy: 0.6969


21/21 [==============================] - 1s 26ms/step - loss: 1.1319 - categorical_accuracy: 0.4919 - val_loss: 0.9464 - val_categorical_accuracy: 0.6839

Epoch 00033: val_categorical_accuracy did not improve from 0.71122

Epoch 00033: ReduceLROnPlateau reducing learning rate to 0.00011764899536501615.
Epoch 34/200
21/21 [==============================] - 1s 26ms/step - loss: 1.1303 - categorical_accuracy: 0.4985 - val_loss: 0.9582 - val_categorical_accuracy: 0.6744

Epoch 00034: val_categorical_accuracy did not improve from 0.71122
Epoch 35/200
21/21 [==============================] - 1s 26ms/step - loss: 1.1370 - categorical_accuracy: 0.4904 - val_loss: 0.9592 - val_categorical_accuracy: 0.6704

Epoch 00035: val_categorical_accuracy did not improve from 0.71122
Epoch 36/200
21/21 [==============================] - 1s 27ms/step - loss: 1.1382 - categorical_accuracy: 0.4911 - val_loss: 0.9578 - val_categorical_accuracy: 0.6719

Epoch 00036: val_categorical_accuracy did not improve from

## Remove first Dropout layer and Add Dense layer

In [193]:
def create_toplayers():
    """Create top layer using inceptionb resnet v2 base"""
    # Create input layer based on the output of the convolutional base
    lyr_input = Input(shape=(base.layers[-1].output.shape.as_list()[1], ))

    # Add Dense
    lyr_dense1 = Dense(1024, activation='relu')(lyr_input)
    lyr_dense2 = Dense(512, activation='relu')(lyr_dense1)
    lyr_dense3 = Dense(256, activation='relu')(lyr_dense2)


    # Create output layer
    output = Dense(5, activation='softmax')(lyr_dense3)

    model = Model(inputs=[lyr_input], outputs=[output])
    model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
                  metrics = ['categorical_accuracy'])
    return model

In [194]:
batch_size = 512
train_gen = data_generator(train_labels, train_data, batch_size)
val_gen = data_generator(val_labels, val_data, batch_size)
train_steps = len(train_labels) // batch_size
val_steps = len(val_labels) // batch_size

In [195]:
model_fp = 'tune/add_more_dense1.h5'
checkpoint = ModelCheckpoint(model_fp,
                             monitor='val_categorical_accuracy',
                             verbose=1, save_best_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_categorical_accuracy',
                                   factor=0.8,
                                   patience=3,
                                   cooldown=3,
                                   verbose=1,
                                   min_lr=0.0001)
early = EarlyStopping(monitor="val_categorical_accuracy", 
                      verbose=1,
                      patience=30)

In [196]:
model = create_toplayers()
model.fit(train_gen,
          steps_per_epoch=train_steps,
          validation_data = val_gen,
          validation_steps = val_steps,
          epochs=200,
          callbacks=[checkpoint, reduceLROnPlat])

Epoch 1/200
21/21 [==============================] - 1s 30ms/step - loss: 2.2193 - categorical_accuracy: 0.2688 - val_loss: 1.0583 - val_categorical_accuracy: 0.7367

Epoch 00001: val_categorical_accuracy improved from -inf to 0.73667, saving model to tune/add_more_dense1.h5
Epoch 2/200
21/21 [==============================] - 0s 25ms/step - loss: 1.4752 - categorical_accuracy: 0.3197 - val_loss: 1.2666 - val_categorical_accuracy: 0.0794

Epoch 00002: val_categorical_accuracy did not improve from 0.73667
Epoch 3/200
21/21 [==============================] - 1s 25ms/step - loss: 1.4306 - categorical_accuracy: 0.3454 - val_loss: 1.3729 - val_categorical_accuracy: 0.1083

Epoch 00003: val_categorical_accuracy did not improve from 0.73667
Epoch 4/200
21/21 [==============================] - 1s 25ms/step - loss: 1.3805 - categorical_accuracy: 0.3651 - val_loss: 1.2320 - val_categorical_accuracy: 0.6209

Epoch 00004: val_categorical_accuracy did not improve from 0.73667

Epoch 00004: ReduceLR


Epoch 00033: val_categorical_accuracy did not improve from 0.75299
Epoch 34/200
21/21 [==============================] - 1s 25ms/step - loss: 1.1128 - categorical_accuracy: 0.5011 - val_loss: 1.1628 - val_categorical_accuracy: 0.5171

Epoch 00034: val_categorical_accuracy did not improve from 0.75299
Epoch 35/200
21/21 [==============================] - 0s 25ms/step - loss: 1.1177 - categorical_accuracy: 0.5004 - val_loss: 0.8659 - val_categorical_accuracy: 0.7087

Epoch 00035: val_categorical_accuracy did not improve from 0.75299

Epoch 00035: ReduceLROnPlateau reducing learning rate to 0.0002621440216898918.
Epoch 36/200
21/21 [==============================] - 1s 25ms/step - loss: 1.0992 - categorical_accuracy: 0.5114 - val_loss: 1.0226 - val_categorical_accuracy: 0.5788

Epoch 00036: val_categorical_accuracy did not improve from 0.75299
Epoch 37/200
21/21 [==============================] - 1s 26ms/step - loss: 1.0885 - categorical_accuracy: 0.5191 - val_loss: 1.2719 - val_categori


Epoch 00066: val_categorical_accuracy did not improve from 0.75299
Epoch 67/200
21/21 [==============================] - 1s 25ms/step - loss: 0.9863 - categorical_accuracy: 0.5701 - val_loss: 1.0186 - val_categorical_accuracy: 0.5801

Epoch 00067: val_categorical_accuracy did not improve from 0.75299
Epoch 68/200
21/21 [==============================] - 1s 25ms/step - loss: 0.9800 - categorical_accuracy: 0.5721 - val_loss: 1.0780 - val_categorical_accuracy: 0.5358

Epoch 00068: val_categorical_accuracy did not improve from 0.75299
Epoch 69/200
21/21 [==============================] - 1s 25ms/step - loss: 0.9902 - categorical_accuracy: 0.5712 - val_loss: 0.9081 - val_categorical_accuracy: 0.6585

Epoch 00069: val_categorical_accuracy did not improve from 0.75299
Epoch 70/200
21/21 [==============================] - 1s 25ms/step - loss: 0.9933 - categorical_accuracy: 0.5657 - val_loss: 0.9747 - val_categorical_accuracy: 0.5942

Epoch 00070: val_categorical_accuracy did not improve from 


Epoch 00101: val_categorical_accuracy did not improve from 0.75299
Epoch 102/200
21/21 [==============================] - 1s 25ms/step - loss: 0.9213 - categorical_accuracy: 0.6021 - val_loss: 0.9399 - val_categorical_accuracy: 0.6185

Epoch 00102: val_categorical_accuracy did not improve from 0.75299
Epoch 103/200
21/21 [==============================] - 1s 25ms/step - loss: 0.9210 - categorical_accuracy: 0.6050 - val_loss: 1.1070 - val_categorical_accuracy: 0.5060

Epoch 00103: val_categorical_accuracy did not improve from 0.75299
Epoch 104/200
21/21 [==============================] - 1s 25ms/step - loss: 0.9212 - categorical_accuracy: 0.6103 - val_loss: 0.8515 - val_categorical_accuracy: 0.6870

Epoch 00104: val_categorical_accuracy did not improve from 0.75299
Epoch 105/200
21/21 [==============================] - 0s 25ms/step - loss: 0.9238 - categorical_accuracy: 0.6040 - val_loss: 0.8389 - val_categorical_accuracy: 0.6939

Epoch 00105: val_categorical_accuracy did not improve f

21/21 [==============================] - 1s 25ms/step - loss: 0.8608 - categorical_accuracy: 0.6426 - val_loss: 1.0010 - val_categorical_accuracy: 0.5499

Epoch 00136: val_categorical_accuracy did not improve from 0.75299
Epoch 137/200
21/21 [==============================] - 1s 25ms/step - loss: 0.8684 - categorical_accuracy: 0.6418 - val_loss: 0.9274 - val_categorical_accuracy: 0.6466

Epoch 00137: val_categorical_accuracy did not improve from 0.75299
Epoch 138/200
21/21 [==============================] - 1s 25ms/step - loss: 0.8760 - categorical_accuracy: 0.6258 - val_loss: 0.8733 - val_categorical_accuracy: 0.6580

Epoch 00138: val_categorical_accuracy did not improve from 0.75299
Epoch 139/200
21/21 [==============================] - 1s 25ms/step - loss: 0.8683 - categorical_accuracy: 0.6366 - val_loss: 0.8720 - val_categorical_accuracy: 0.6829

Epoch 00139: val_categorical_accuracy did not improve from 0.75299
Epoch 140/200
21/21 [==============================] - 1s 26ms/step - 

21/21 [==============================] - 1s 25ms/step - loss: 0.8255 - categorical_accuracy: 0.6585 - val_loss: 1.1963 - val_categorical_accuracy: 0.4764

Epoch 00171: val_categorical_accuracy did not improve from 0.75299
Epoch 172/200
21/21 [==============================] - 1s 25ms/step - loss: 0.8270 - categorical_accuracy: 0.6614 - val_loss: 0.8418 - val_categorical_accuracy: 0.6885

Epoch 00172: val_categorical_accuracy did not improve from 0.75299
Epoch 173/200
21/21 [==============================] - 1s 25ms/step - loss: 0.8131 - categorical_accuracy: 0.6595 - val_loss: 0.9062 - val_categorical_accuracy: 0.6518

Epoch 00173: val_categorical_accuracy did not improve from 0.75299
Epoch 174/200
21/21 [==============================] - 0s 25ms/step - loss: 0.8145 - categorical_accuracy: 0.6562 - val_loss: 1.1492 - val_categorical_accuracy: 0.4879

Epoch 00174: val_categorical_accuracy did not improve from 0.75299
Epoch 175/200
21/21 [==============================] - 0s 25ms/step - 

In [198]:
model = load_model(model_fp)
print('train loss {} accuracy {}'.format(*model.evaluate(train_gen, steps=train_steps)))

21/21 [==============================] - 0s 9ms/step - loss: 1.2558 - categorical_accuracy: 0.4285
train loss 1.2557694911956787 accuracy 0.4284784197807312


In [199]:
model = load_model(model_fp)
print('train loss {} accuracy {}'.format(*model.evaluate(val_gen, steps=val_steps)))

34/34 [==============================] - 0s 9ms/step - loss: 0.8217 - categorical_accuracy: 0.7533
train loss 0.8216578364372253 accuracy 0.7533317804336548


## Remove first Dropout layer and Add Dense layer, Reduce Batch size

In [206]:
def create_toplayers():
    """Create top layer using inceptionb resnet v2 base"""
    # Create input layer based on the output of the convolutional base
    lyr_input = Input(shape=(base.layers[-1].output.shape.as_list()[1], ))

    # Add Dense
    lyr_dense1 = Dense(1024, activation='relu')(lyr_input)
    lyr_dense2 = Dense(512, activation='relu')(lyr_dense1)
    lyr_dense3 = Dense(256, activation='relu')(lyr_dense2)


    # Create output layer
    output = Dense(5, activation='softmax')(lyr_dense3)

    model = Model(inputs=[lyr_input], outputs=[output])
    model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
                  metrics = ['categorical_accuracy'])
    return model

In [212]:
batch_size = 1024
train_gen = data_generator(train_labels, train_data, batch_size)
val_gen = data_generator(val_labels, val_data, batch_size)
train_steps = len(train_labels) // batch_size
val_steps = len(val_labels) // batch_size

In [213]:
model_fp = 'tune/add_more_dense2.h5'
checkpoint = ModelCheckpoint(model_fp,
                             monitor='val_categorical_accuracy',
                             verbose=1, save_best_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_categorical_accuracy',
                                   factor=0.8,
                                   patience=3,
                                   cooldown=3,
                                   verbose=1,
                                   min_lr=0.0001)
early = EarlyStopping(monitor="val_categorical_accuracy", 
                      verbose=1,
                      patience=30)

In [214]:
model = create_toplayers()
model.fit(train_gen,
          steps_per_epoch=train_steps,
          validation_data = val_gen,
          validation_steps = val_steps,
          epochs=200,
          callbacks=[checkpoint, reduceLROnPlat])

Epoch 1/200
10/10 [==============================] - 1s 68ms/step - loss: 2.5835 - categorical_accuracy: 0.3044 - val_loss: 1.1426 - val_categorical_accuracy: 0.7367

Epoch 00001: val_categorical_accuracy improved from -inf to 0.73667, saving model to tune/add_more_dense2.h5
Epoch 2/200
10/10 [==============================] - 0s 54ms/step - loss: 1.4937 - categorical_accuracy: 0.3297 - val_loss: 1.1731 - val_categorical_accuracy: 0.7334

Epoch 00002: val_categorical_accuracy did not improve from 0.73667
Epoch 3/200
10/10 [==============================] - 0s 54ms/step - loss: 1.4498 - categorical_accuracy: 0.3251 - val_loss: 1.0595 - val_categorical_accuracy: 0.7353

Epoch 00003: val_categorical_accuracy did not improve from 0.73667
Epoch 4/200
10/10 [==============================] - 1s 55ms/step - loss: 1.4858 - categorical_accuracy: 0.3421 - val_loss: 1.1796 - val_categorical_accuracy: 0.6621

Epoch 00004: val_categorical_accuracy did not improve from 0.73667

Epoch 00004: ReduceLR


Epoch 00033: val_categorical_accuracy did not improve from 0.73667
Epoch 34/200
10/10 [==============================] - 0s 54ms/step - loss: 1.1563 - categorical_accuracy: 0.4845 - val_loss: 1.1724 - val_categorical_accuracy: 0.4706

Epoch 00034: val_categorical_accuracy did not improve from 0.73667

Epoch 00034: ReduceLROnPlateau reducing learning rate to 0.00020971521735191345.
Epoch 35/200
10/10 [==============================] - 1s 55ms/step - loss: 1.1428 - categorical_accuracy: 0.4874 - val_loss: 0.9659 - val_categorical_accuracy: 0.6683

Epoch 00035: val_categorical_accuracy did not improve from 0.73667
Epoch 36/200
10/10 [==============================] - 1s 54ms/step - loss: 1.1442 - categorical_accuracy: 0.4834 - val_loss: 1.0467 - val_categorical_accuracy: 0.5920

Epoch 00036: val_categorical_accuracy did not improve from 0.73667
Epoch 37/200
10/10 [==============================] - 1s 55ms/step - loss: 1.1397 - categorical_accuracy: 0.4937 - val_loss: 0.9076 - val_categor

10/10 [==============================] - 1s 55ms/step - loss: 1.0756 - categorical_accuracy: 0.5217 - val_loss: 0.8955 - val_categorical_accuracy: 0.6994

Epoch 00067: val_categorical_accuracy did not improve from 0.73667
Epoch 68/200
10/10 [==============================] - 1s 55ms/step - loss: 1.0888 - categorical_accuracy: 0.5178 - val_loss: 1.0443 - val_categorical_accuracy: 0.5675

Epoch 00068: val_categorical_accuracy did not improve from 0.73667
Epoch 69/200
10/10 [==============================] - 0s 53ms/step - loss: 1.0841 - categorical_accuracy: 0.5261 - val_loss: 0.9508 - val_categorical_accuracy: 0.6447

Epoch 00069: val_categorical_accuracy did not improve from 0.73667
Epoch 70/200
10/10 [==============================] - 0s 54ms/step - loss: 1.0685 - categorical_accuracy: 0.5306 - val_loss: 1.0663 - val_categorical_accuracy: 0.5506

Epoch 00070: val_categorical_accuracy did not improve from 0.73667
Epoch 71/200
10/10 [==============================] - 1s 55ms/step - loss

10/10 [==============================] - 1s 56ms/step - loss: 1.0645 - categorical_accuracy: 0.5317 - val_loss: 1.0246 - val_categorical_accuracy: 0.5708

Epoch 00102: val_categorical_accuracy did not improve from 0.73667
Epoch 103/200
10/10 [==============================] - 1s 55ms/step - loss: 1.0366 - categorical_accuracy: 0.5444 - val_loss: 0.8668 - val_categorical_accuracy: 0.7059

Epoch 00103: val_categorical_accuracy did not improve from 0.73667
Epoch 104/200
10/10 [==============================] - 1s 56ms/step - loss: 1.0513 - categorical_accuracy: 0.5416 - val_loss: 1.0411 - val_categorical_accuracy: 0.5609

Epoch 00104: val_categorical_accuracy did not improve from 0.73667
Epoch 105/200
10/10 [==============================] - 1s 56ms/step - loss: 1.0494 - categorical_accuracy: 0.5436 - val_loss: 0.8713 - val_categorical_accuracy: 0.6951

Epoch 00105: val_categorical_accuracy did not improve from 0.73667
Epoch 106/200
10/10 [==============================] - 1s 55ms/step - 


Epoch 00136: val_categorical_accuracy did not improve from 0.73794
Epoch 137/200
10/10 [==============================] - 1s 56ms/step - loss: 1.0186 - categorical_accuracy: 0.5540 - val_loss: 0.9306 - val_categorical_accuracy: 0.6506

Epoch 00137: val_categorical_accuracy did not improve from 0.73794
Epoch 138/200
10/10 [==============================] - 1s 55ms/step - loss: 1.0144 - categorical_accuracy: 0.5567 - val_loss: 1.0315 - val_categorical_accuracy: 0.5569

Epoch 00138: val_categorical_accuracy did not improve from 0.73794
Epoch 139/200
10/10 [==============================] - 1s 55ms/step - loss: 1.0027 - categorical_accuracy: 0.5622 - val_loss: 0.8252 - val_categorical_accuracy: 0.7264

Epoch 00139: val_categorical_accuracy did not improve from 0.73794
Epoch 140/200
10/10 [==============================] - 1s 55ms/step - loss: 1.0290 - categorical_accuracy: 0.5507 - val_loss: 1.0551 - val_categorical_accuracy: 0.5530

Epoch 00140: val_categorical_accuracy did not improve f

10/10 [==============================] - 1s 59ms/step - loss: 0.9973 - categorical_accuracy: 0.5624 - val_loss: 1.0037 - val_categorical_accuracy: 0.5864

Epoch 00171: val_categorical_accuracy did not improve from 0.73794
Epoch 172/200
10/10 [==============================] - 1s 59ms/step - loss: 0.9902 - categorical_accuracy: 0.5751 - val_loss: 1.0343 - val_categorical_accuracy: 0.5576

Epoch 00172: val_categorical_accuracy did not improve from 0.73794
Epoch 173/200
10/10 [==============================] - 1s 62ms/step - loss: 0.9860 - categorical_accuracy: 0.5745 - val_loss: 0.9210 - val_categorical_accuracy: 0.6502

Epoch 00173: val_categorical_accuracy did not improve from 0.73794
Epoch 174/200
10/10 [==============================] - 1s 60ms/step - loss: 0.9809 - categorical_accuracy: 0.5763 - val_loss: 1.0362 - val_categorical_accuracy: 0.5519

Epoch 00174: val_categorical_accuracy did not improve from 0.73794
Epoch 175/200
10/10 [==============================] - 1s 57ms/step - 

In [215]:
model = load_model(model_fp)
print('train loss {} accuracy {}'.format(*model.evaluate(train_gen, steps=train_steps)))

10/10 [==============================] - 0s 17ms/step - loss: 1.0655 - categorical_accuracy: 0.5210
train loss 1.0655215978622437 accuracy 0.52099609375


In [216]:
model = load_model(model_fp)
print('train loss {} accuracy {}'.format(*model.evaluate(val_gen, steps=val_steps)))

17/17 [==============================] - 0s 18ms/step - loss: 0.8140 - categorical_accuracy: 0.7383
train loss 0.8140085935592651 accuracy 0.73828125


## Remove first Dropout layer and Add Dense layer, Reduce Batch size perform sampling using mean as n_samples

In [ ]:
def create_toplayers():
    """Create top layer using inceptionb resnet v2 base"""
    # Create input layer based on the output of the convolutional base
    lyr_input = Input(shape=(base.layers[-1].output.shape.as_list()[1], ))

    # Add Dense
    lyr_dense1 = Dense(1024, activation='relu')(lyr_input)
    lyr_dense2 = Dense(512, activation='relu')(lyr_dense1)
    lyr_dense3 = Dense(256, activation='relu')(lyr_dense2)


    # Create output layer
    output = Dense(5, activation='softmax')(lyr_dense3)

    model = Model(inputs=[lyr_input], outputs=[output])
    model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy',
                  metrics = ['categorical_accuracy'])
    return model

In [ ]:
batch_size = 1024
train_gen = data_generator(train_labels, train_data, batch_size)
val_gen = data_generator(val_labels, val_data, batch_size)
train_steps = len(train_labels) // batch_size
val_steps = len(val_labels) // batch_size

In [ ]:
model_fp = 'tune/add_more_dense3.h5'
checkpoint = ModelCheckpoint(model_fp,
                             monitor='val_categorical_accuracy',
                             verbose=1, save_best_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_categorical_accuracy',
                                   factor=0.8,
                                   patience=3,
                                   cooldown=3,
                                   verbose=1,
                                   min_lr=0.0001)
early = EarlyStopping(monitor="val_categorical_accuracy", 
                      verbose=1,
                      patience=30)

In [ ]:
model = create_toplayers()
model.fit(train_gen,
          steps_per_epoch=train_steps,
          validation_data = val_gen,
          validation_steps = val_steps,
          epochs=200,
          callbacks=[checkpoint, reduceLROnPlat])

In [ ]:
model = load_model(model_fp)
print('train loss {} accuracy {}'.format(*model.evaluate(train_gen, steps=train_steps)))

In [ ]:
model = load_model(model_fp)
print('train loss {} accuracy {}'.format(*model.evaluate(val_gen, steps=val_steps)))